In [1]:
import cv2
import tensorflow as tf
import mediapipe as mp
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras import optimizers  # Optimizer that we will use to train the model
from tensorflow.keras.models import Sequential  # Allows us to make sequential neural networks
from tensorflow.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D  # Layers for performing convolutions
from tensorflow.keras import backend as K  # If there's a Keras session, we close it to keep everything clean

model_path = '/Users/luna/Desktop/carpeta sin título/hand identifier/Model.h5'
weights_path = '/Users/luna/Desktop/carpeta sin título/hand identifier/weights.h5'
cnn = load_model(model_path)
cnn.load_weights(weights_path)

test_directory = '/Users/luna/Desktop/carpeta sin título/hand identifier/photos/test'
test_images = os.listdir(test_directory)
print("Names: ", test_images)

# Reading the camera
cap = cv2.VideoCapture(0)

#---------------------------- Create an object that will store the detection and tracking of hands ------------
hands_class = mp.solutions.hands
hands = hands_class.Hands()

drawing = mp.solutions.drawing_utils  # Draws the 21 key points of the hand

while True:
    ret, frame = cap.read()
    color = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    copy = frame.copy()
    result = hands.process(color)
    positions = []  # In this list, we will store the coordinates of the points
    #print(result.multi_hand_landmarks)

    if result.multi_hand_landmarks:  # If there is something in the results, we enter the if
        for hand in result.multi_hand_landmarks:  # We look for the hand within the list of hands given by the descriptor
            for id, lm in enumerate(hand.landmark):  # We will obtain the information of each hand found by the ID
                height, width, c = frame.shape  # We extract the width and height of the frames to multiply them by the proportion
                corx, cory = int(lm.x * width), int(lm.y * height)  # We extract the location of each point that belongs to the hand in coordinates
                positions.append([id, corx, cory])
                drawing.draw_landmarks(frame, hand, hands_class.HAND_CONNECTIONS)
            if positions:
                central_point = positions[9]
                x1, y1 = (central_point[1] - 80), (central_point[2] - 80)
                width, height = (x1 + 80), (y1 + 80)
                x2, y2 = x1 + width, y1 + height
                hand_region = copy[y1:y2, x1:x2]
                hand_region = cv2.resize(hand_region, (200, 200), interpolation=cv2.INTER_CUBIC)  # Resize the photos
                x = img_to_array(hand_region)  # Convert the image to an array
                x = np.expand_dims(x, axis=0)
                prediction = cnn.predict(x)  # It will be a 2-dimensional array, where it will put 1 in the class it believes correct
                result = prediction[0]  # [1,0] | [0, 1]
                response = np.argmax(result)  # It gives us the index of the highest value 0 | 1
                if response == 1:
                    print(prediction, result)
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)
                    cv2.putText(frame, '{}'.format(test_images[0]), (x1, y1 - 5), 1, 1.3, (0, 255, 0), 1, cv2.LINE_AA)
                elif response == 0:
                    print(prediction, result)
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 3)
                    cv2.putText(frame, '{}'.format(test_images[1]), (x1, y1 - 5), 1, 1.3, (0, 0, 255), 1, cv2.LINE_AA)

    cv2.imshow("Video", frame)
    k = cv2.waitKey(1)
    if k == 27:
        break
cap.release()
cv2.destroyAllWindows()

2024-02-17 12:14:03.775886: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Names:  ['.DS_Store', 'left hand', 'right hand']


I0000 00:00:1708168453.692720       1 gl_context.cc:344] GL version: 2.1 (2.1 INTEL-20.6.4), renderer: Intel(R) Iris(TM) Plus Graphics 645
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


1/1 [==============================] - 0s 153ms/step
[[1. 0.]] [1. 0.]
1/1 [==============================] - 0s 46ms/step
[[1. 0.]] [1. 0.]
1/1 [==============================] - 0s 37ms/step
[[1. 0.]] [1. 0.]
1/1 [==============================] - 0s 37ms/step
[[1. 0.]] [1. 0.]
1/1 [==============================] - 0s 35ms/step
[[1. 0.]] [1. 0.]
1/1 [==============================] - 0s 35ms/step
[[0. 1.]] [0. 1.]
1/1 [==============================] - 0s 34ms/step
[[0. 1.]] [0. 1.]
1/1 [==============================] - 0s 35ms/step
[[1. 0.]] [1. 0.]
1/1 [==============================] - 0s 35ms/step
[[1. 0.]] [1. 0.]
1/1 [==============================] - 0s 35ms/step
[[1. 0.]] [1. 0.]
1/1 [==============================] - 0s 34ms/step
[[1. 0.]] [1. 0.]
1/1 [==============================] - 0s 36ms/step
[[1. 0.]] [1. 0.]
1/1 [==============================] - 0s 35ms/step
[[1. 0.]] [1. 0.]
1/1 [==============================] - 0s 36ms/step
[[1. 0.]] [1. 0.]
1/1 [==============